In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import os
import yaml
from huggingface_hub import login

In [20]:
os.chdir('/home/mozerski/TM-2024/')

with open("hf_token.yaml", "r") as file:
    token_config = yaml.safe_load(file)
token = token_config.get("hf_token", None)
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /llm/huggingface/token
Login successful


### Just with fine_tuned_lora_llama (DOESNT WORK)

In [84]:
model_dir = "fine_tuned_lora_llama"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the base model
model = AutoModelForCausalLM.from_pretrained(model_dir)

model = PeftModel.from_pretrained(model, model_dir)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Fine-tuned model and tokenizer loaded successfully!")


RuntimeError: Error(s) in loading state_dict for LlamaForCausalLM:
	size mismatch for model.embed_tokens.weight: copying a param with shape torch.Size([128260, 2048]) from checkpoint, the shape in current model is torch.Size([128256, 2048]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([128260, 2048]) from checkpoint, the shape in current model is torch.Size([128256, 2048]).

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=150, temperature=0.1, top_k=50, top_p=0.9):
    """
    Generate text using the fine-tuned model.
    """
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"], 
        max_length=max_length, 
        temperature=temperature,
        #top_k=10,
        #top_p=0.9,
        #repetition_penalty=1.15,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "<|startoftext|>[INST] Describe Caleb [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

### With model = PeftModel.from_pretrained(model, 'fine_tuned_lora_llama')

In [21]:
# Prepare data
model_name = "meta-llama/Llama-3.2-1B"
folder_path = "./DATASET"

# Load the Fine-tuned Model for Generation
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=token,
    torch_dtype=torch.float16,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Add special tokens to tokenizer and resize model embeddings
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=token,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Add special tokens and resize tokenizer
special_tokens_dict = {'additional_special_tokens': ['<|startoftext|>', '[INST]', '[/INST]', '<|endoftext|>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# Resize model embeddings to match new tokenizer size
model.resize_token_embeddings(len(tokenizer))

# Load LoRA adapters
model = PeftModel.from_pretrained(model, 'fine_tuned_lora_llama')
model.to(device)
model.eval()

# Function to Generate Text
def generate_text(model, tokenizer, prompt, max_length=50, temperature=0.9, top_k=50, top_p=0.9):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            #temperature=temperature,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            #top_k=top_k,
            #top_p=top_p,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the Fine-tuned Model
prompt = "<|startoftext|>[INST] Describe Axel [/INST]"
generated_text = generate_text(model, tokenizer, prompt)
print("Generated Text:")
print(generated_text)

Generated Text:
 Describe Axel  Axel is a blockchain developer with expertise in building decentralized applications (DApps). He has worked on projects involving Ethereum and smart contracts. Axel is passionate about leveraging blockchain for innovative solutions.  kalan is passionate about innovative use
